In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
lstm=nn.LSTM(3,3)
inputs=[torch.rand(1,3)for _ in range(5)]

hidden=(torch.randn(1,1,3),
       torch.randn(1,1,3))

for i in inputs:
    out,hidden=lstm(i.view(1,1,-1),hidden)



In [3]:
inputs=torch.cat(inputs).view(len(inputs),1,-1)
hidden=(torch.randn(1,1,3),torch.randn(1,1,3))
out,hidden=lstm(inputs,hidden)
print(out)
print(hidden)

tensor([[[ 0.0672,  0.2668, -0.2215]],

        [[ 0.0933,  0.1122, -0.1220]],

        [[ 0.1313, -0.0614, -0.1112]],

        [[ 0.0404, -0.1227, -0.1019]],

        [[ 0.0385, -0.2224, -0.0812]]], grad_fn=<StackBackward>)
(tensor([[[ 0.0385, -0.2224, -0.0812]]], grad_fn=<StackBackward>), tensor([[[ 0.0797, -0.3901, -0.4256]]], grad_fn=<StackBackward>))


# 用LSTM来进行词性标注

In [6]:
def prepare_sequence(seq,to_ix):
    idxs=[to_ix[w] for w in seq]
    return torch.tensor(idxs,dtype=torch.long)
training_data=[
    ('The dog ate the apple'.split(),['DET','NN','V','DET','NN']),
    ('Everybody read that book'.split(),['NN','V','DET','NN'])
]
word_to_ix={}
for sent,tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word]=len(word_to_ix)
print(word_to_ix)
tag_to_ix={"DET":0,"NN":1,"V":2}

EMBEDDING_DIM=6
HIDDEN_DIM=6


{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [13]:
class LSTMTagger(nn.Module):
    def __init__(self,embedding_dim,hidden_dim,vocab_size,tagset_size):
        super(LSTMTagger,self).__init__()
        self.hidden_dim=hidden_dim
        
        self.word_embeddings=nn.Embedding(vocab_size,embedding_dim)
        self.lstm=nn.LSTM(embedding_dim,hidden_dim)
        self.hidden2tag=nn.Linear(hidden_dim,tagset_size)
        self.hidden=self.init_hidden()
        
    def init_hidden(self):
        return (torch.zeros(1,1,self.hidden_dim),(torch.zeros(1,1,self.hidden_dim))
    
    def forward(self,sentence):
        embeds=self.word_embeddings(sentence)
        lstm_out,self.hidden=self.lstm(embeds.view(len(sentence),1,-1),self.hidden)
        tag_space=self.hidden2tag(lstm_out.view(len(sentence),-1))
        tag_scores=F.log_softmax(tag_space,dim=1)
        return tag_scores

SyntaxError: invalid syntax (<ipython-input-13-aca781da41a5>, line 14)

In [14]:
model=LSTMTagger(EMBEDDING_DIM,HIDDEN_DIM,len(word_to_ix),len(tag_to_ix))
loss_function=nn.NULLLoss()
optimizer=optim.SGD(model.parameters(),lr=0.1)


NameError: name 'LSTMTagger' is not defined

In [17]:
with torch.no_grad():
    inputs=prepare_sequence(training_data[0][0],word_to_ix)
    tag_scores=model(inputs)
    print(tag_scores)
for epoch in range(300):
    for sentence,tags in training_data:
        model.zero_grad()
        model.hidden=model.init_hidden()
        sentence_in=prepare_sequence(sentence,word_to_ix)
        targets=prepare_sequence(tags,tag_to_ix)
        tag_scores=model(sentence_in)
        loss=loss_function(tag_scores,targets)
        loss.backward()
        optimizer.step()
        
        
with torch.no_grad():
    inputs=prepare_sequence(training_data[0][0],word_to_ix)
    tag_scores=model(inputs)
    print(tag_scores)

NameError: name 'model' is not defined